## install extra libs

In [ ]:
!pip install diffusers

## Import libs

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, Conversation, AutoModelForSequenceClassification

# Image Generation
from diffusers import DiffusionPipeline
import matplotlib as plt
import IPython.display as display

import torch
import torch
import torch.nn.functional as F

# Clear CUDA
import gc 

## Example | sentiment-analysis

In [ ]:
classifier = pipeline("sentiment-analysis")



In [ ]:
res = classifier("Hi, nice to meet you!")
print(res)

## Example | text-generation

In [ ]:
generator = pipeline("text-generation")

In [ ]:
res = generator(
    "Have the machines imagination?",
    max_length=128,
    num_return_sequences=1,
)

print(res[0]["generated_text"])

## Example | zero-shot-classification

In [ ]:
classifier = pipeline("zero-shot-classification")

In [ ]:
res = classifier(
    "I'm going to see some local sightseeing",
    candidate_labels = ["education", "travel", "buisness"],
)

print(res)

## Example | conversational

In [ ]:
chatbot = pipeline("conversational", model="microsoft/DialoGPT-medium")

In [ ]:
is_working = False
conversation = Conversation()

while is_working:
    inputs = input("You: ")
    
    if inputs == "stop":
        break
    
    conversation.add_user_input(inputs)
    conversation = chatbot(conversation)
    
    print("Bot: ", conversation.generated_responses[-1])

In [ ]:
conversation

```
user >> Going to the movies tonight - any suggestions? 
bot >> The Big Lebowski 
user >> Is it an action movie? 
bot >> It's a comedy. 
user >> I do not like comedy 
bot >> I do not like you 
user >> but why? 
bot >> I don't like you 
user >> how is it going 
bot >> It's going well. ```

## Example | Image generation

In [ ]:
pipe = DiffusionPipeline.from_pretrained("google/ddpm-cifar10-32")

In [ ]:
res = pipe()

In [ ]:
# Display the image in the notebook
display.display(res[0][0].resize((256, 256)))

## Example | Image generation with prompt

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
pipe = DiffusionPipeline.from_pretrained("dreamlike-art/dreamlike-photoreal-2.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")

In [ ]:
#pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)
# pipe.to("cuda")

pipe.enable_model_cpu_offload()

In [ ]:
prompt = "A cute black kitty with blue eyes" 
images = pipe(prompt=prompt).images[0]

In [ ]:
images

In [ ]:
images.save("_".join(prompt.split(" ")) + ".png")

In [ ]:
torch.cuda.empty_cache()
gc.collect()

## Tokinazer & Model

In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
inputs = [
    "This film is quite boring",
    "Hi nice to meet you on this wonderful day!"
]

res = classifier(inputs)
print(res)

In [ ]:
res = tokenizer(inputs)
res

In [ ]:
tokens = tokenizer.tokenize(inputs)
tokens

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

In [ ]:
decoded_res = tokenizer.decode(ids)
decoded_res

## PyTorch 

In [ ]:
batch = tokenizer(inputs, padding=True, truncation=True, max_length=512, return_tensors="pt")
print(batch)

In [ ]:
with torch.no_grad():
    outputs = model(**batch)
    predictions = F.softmax(outputs.logits, dim=1)
    labels = torch.argmax(predictions, dim=1)

In [ ]:
outputs

In [ ]:
predictions

In [ ]:
labels

## Saving model & tokenizer

In [ ]:
!pwd

In [ ]:
save_dir = "/kaggle/working"

tokenizer.save_pretrained(save_dir)
model.save_pretrained(save_dir)

In [ ]:
tokenizer_saved = AutoTokenizer.from_pretrained(save_dir)
model_saved = AutoModelForSequenceClassification.from_pretrained(save_dir) 

In [ ]:
nan